In [2]:
import os

import findspark
findspark.init("/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3")
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.types as T
import pyspark.sql.functions as F

# warehouse_location points to the default location for managed databases and tables
os.environ["PYSPARK_PYTHON"]="/home/pc/TestJupyter/opt/spark-3.3.0/venv-spark/bin/python39"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.3.0  pyspark-shell'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.driver.memory", "60g")\
    .config("spark.sql.warehouse.dir", "hdfs://10.123.51.78:8000/user/hive/warehouse") \
    .config("spark.sql.catalogImplementation", "hive")\
    .config("spark.executor.cores", 18) \
    .config("hive.metastore.uris", "thrift://g2.bigtop.it:9083")\
    .enableHiveSupport() \
    .getOrCreate()

def read_avro(path):
    return spark.read.format("avro").load(path)

def write_avro(df, path):
    return df.write.format("avro").save(path)

:: loading settings :: url = jar:file:/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pc/.ivy2/cache
The jars for the packages stored in: /home/pc/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-85210af3-3868-4e0b-b4cf-a69292d3ac49;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.0 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 127ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.3.0 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |

22/10/11 05:56:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/11 05:56:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/11 05:56:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/11 05:56:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
bi = spark.read.text("/home/pc/data/parsed_data/1parsed-extract-enwiki.txt")
bm = spark.read.text("/home/pc/data/parsed_data/2parsed-extract-mswiki.txt")
bc = spark.read.text("/home/pc/data/parsed_data/3parsed-extract-zhwiki.txt")

print(bi.count())
print(bm.count())
print(bc.count())

# bi.write.mode("overwrite").format("avro").save("output/step1-__BI__.avro")
# bm.write.mode("overwrite").format("avro").save("output/step1-__BM__.avro")
# bc.write.mode("overwrite").format("avro").save("output/step1-__BC__.avro")

768426
707666


2495291


22/10/11 14:51:46 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-3884340c-b0b8-4841-b38f-ab38531b2f8d. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-3884340c-b0b8-4841-b38f-ab38531b2f8d
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1206)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:374)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1$adapted(DiskBlockManager.scala:370)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach

### STEP 1 : Data preprocessing for those token that will affected by the sentence tokenization
--------------------------------------
##### EMAIL : separated by dot
##### HTML  : inside might have dot
##### URL   : separated by dot
--------------------------------------
##### Some structured notation also need to be cleaned like {}

In [79]:
import findspark
findspark.init("/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3")
import pyspark
from pyspark.sql.functions import lower, col, udf
from pyspark.sql.functions import regexp_replace
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS
from pyspark.sql.types import StringType
from cleantext import clean
import pickle
from dateutil import parser
import os
import shutil, sys
import gc
## no braces
## no == b ==
## no category
## no other source

num = "[0-9]"
px = "[0-9]px"
p1 = "\=(.{1,}?)\= "
p2 = "Category:.*|Kategori:.*"
p3 = "\*(.*?)(?=[ *| , | .|*|,|.])"
p4 = "\{([^}]+)\}"
p5 = r"http\S+"
p6 = "thumb\|.+?(?= )"
p7 = "([^ ])+ : ([^ ])+ |([^ ])+:([^ ])+"

email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
def preprocess_for_token_with_dot_delimeter(df, col_name):
    return df.withColumn(col_name, regexp_replace(col_name, email_pattern, 'EMAIL')) \
             .withColumn(col_name, regexp_replace(col_name, r'http\S+', ''))         \
             .withColumn(col_name, regexp_replace(col_name, r'<[^>]+>', ''))

def pp(df, col_name):
    return df \
    .withColumn(col_name, regexp_replace(col_name, f"{p5}", '')) \
    .withColumn(col_name, regexp_replace(col_name, f"{p4}", '')) \
    .withColumn(col_name, regexp_replace(col_name, f"{p1}", '')) \
    .withColumn(col_name, regexp_replace(col_name, f"{p2}", '')) \
    .withColumn(col_name, regexp_replace(col_name, f"{p3}|{p6}|{p7}|[0-9]", '')) \
    .withColumn(col_name, regexp_replace(col_name, f"{num}|{px}|colspan|right thumb", '')) \
    .withColumn(col_name, lower(col_name)) 
    return preprocess_for_token_with_dot_delimeter(df, col_name)
    
    

In [80]:
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
def preprocess_for_token_with_dot_delimeter(df, col_name):
    return df.withColumn(col_name, regexp_replace(col_name, email_pattern, 'EMAIL')) \
             .withColumn(col_name, regexp_replace(col_name, r'http\S+', ''))         \
             .withColumn(col_name, regexp_replace(col_name, r'<[^>]+>', ''))

In [81]:
bi = spark.read.format("avro").load("output/step1-__BI__.avro")
bm = spark.read.format("avro").load("output/step1-__BM__.avro")
bc = spark.read.format("avro").load("output/step1-__BC__.avro")

bi = bi.repartition(100)
bm = bm.repartition(100)
bc = bc.repartition(100)

bi = bi.withColumnRenamed("value","text")
bm = bm.withColumnRenamed("value","text")
bc = bc.withColumnRenamed("value","text")

In [77]:
bi.sample(0.6, 1234).show(10, False)
## CLICK TO SHOW THE UNCLEANED SENTENCES

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [32]:
pp(bi,"text").sample(0.6, 0.6, 1234).show(10, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK TO SHOW THE CLEANED SENTENCES

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
shutil.rmtree("output/1_ppBC__.avro")

In [82]:
pp(bc,"text").write.mode("overwrite").format("avro").save("output/1_ppBC__.avro")
pp(bm,"text").write.mode("overwrite").format("avro").save("output/1_ppBM__.avro")
pp(bi,"text").write.mode("overwrite").format("avro").save("output/1_ppBI__.avro")

### STEP 2 : Doing sentence tokenization
--------------------------------------
##### chinese = using 
##### malay = using nltk 
##### enlish = using harvest text
--------------------------------------
##### Some structured notation also need to be cleaned like {}

In [11]:
bc

DataFrame[text: string]

In [83]:
bc = read_avro("output/1_ppBC__.avro")
bm = read_avro("output/1_ppBM__.avro")
bi = read_avro("output/1_ppBI__.avro")

bc = bc.filter(bc.text != "")
bm = bm.filter(bm.text != "")
bi = bi.filter(bi.text != "")

In [ ]:
## sentence tokenize for chinese using harvest-text library

#### Tokenize chinese sentences

In [5]:
bc.sample(0.7, 7).show(5, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK TO SEE THE UNTOKENIZED CHINESE SENTENCE

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|t

In [63]:
from harvesttext import HarvestText
ht = HarvestText()

def chinese_para_to_sentence(text):
    return ht.cut_sentences(text)

getChiS = F.udf(lambda x : chinese_para_to_sentence(x), T.ArrayType(T.StringType()))


def get_df_chiSen(df, colname):
    return df.withColumn(colname, getChiS(F.col(colname)))

In [84]:
df = get_df_chiSen(bc, "text")
df= df.withColumn("text", F.explode(F.col('text')))
df.show(50,False)

# 🆂🅰🅼🅿🅻🅴
## CLICK TO SEE THE TOKENIZED CHINESE SENTENCE

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                                                                           

In [86]:
write_avro(df, "output/2_SPLIT_CHINESE")

#### Tokenize english sentences

In [87]:
from pyspark.sql.functions import *
from nltk.tokenize import sent_tokenize
def get_sentenct_token(t):
    return sent_tokenize(t, "english")

engSentence = udf(lambda t:get_sentenct_token(t),T.ArrayType(T.StringType()))

def get_eng_senDF(df, coln):
    return df.withColumn(coln, engSentence(col(coln)))

In [88]:
df = get_eng_senDF(bi, "text")

In [89]:
df= df.select(F.explode('text'))

In [24]:
bi.show(5, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK TO SHOW THE UNTOKENIZED ENGLISH SENTENCES

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
df.sample(0.5).show(25, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK TO SHOW THE TOKENIZED ENGLISH SENTENCES

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                                                                                                              |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|richmond is noted for his role in the creation of the head start program for disadvantaged children, serving as its first national director.                                                                                                     |
|stable version is licen

In [90]:
write_avro(df, "output/2_ENG_SENTENCES")

#### Tokenize MALAY sentences

In [91]:
df = get_eng_senDF(bm, "text")
df= df.select(F.explode('text'))

In [29]:
bm.show(10, False)

## CLICK TO SHOW UNTOKENIZED BM TEXT

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [30]:
df.show(30, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                                                                               |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|soluk bon-e olya merupakan sebuah kawasan yang terletak di iran.                                                                                                                                                  |
|=                                                                                                                                                  

In [92]:
write_avro(df, "output/2_MALAY_SENTENCES")

In [5]:
c = read_avro("output/2_SPLIT_CHINESE")
m = read_avro("output/2_MALAY_SENTENCES")
i = read_avro("output/2_ENG_SENTENCES")

i = i.repartition(60)

In [6]:
import contractions
from connectPyspark import *
from pyspark.sql.functions import sentences, lit, flatten
from pyspark.sql.types import StringType, ArrayType
from pyspark.sql.functions import col, udf, size, split
import chinese_converter
from cleantext import clean



noContr = udf(lambda x : remove_contractions(x), StringType())
linebreak_pattern = r"((\r\n\t)|[\n\v\t])+"
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
phone_pattern = r"((?:^|(?<=[^\w)]))(((\+?[01])|(\+\d{2}))[ .-]?)?(\(?\d{3,4}\)?/?[ .-]?)?(\d{3}[ .-]?\d{4})(\s?(?:ext\.?|[#x-])\s?\d{2,6})?(?:$|(?=\W)))|\+?\d{4,5}[ .-/]\d{6,9}"
number_pattern = r"(?:^|(?<=[^\w,.]))[+–-]?(([1-9]\d{0,2}(,\d{3})+(\.\d*)?)|([1-9]\d{0,2}([ .]\d{3})+(,\d*)?)|(\d*?[.,]\d+)|\d+)(?:$|(?=\b))"
key_pattern = r"(\w+)(?=:):"
num = "[0-9]"
symbols = """☺|☻|\||\/|♥|\[|♦|♣|\.|♠|•|◘|○|◙|♂|♀|♪|♫|☼|►|◄|↕|‼|¶|\$|§|▬|↨|↑|↓|→|←|∟|↔|▲|▼|#|%|&|,|-|\'|:|;|<|=|>|@|]|_|`|¢|£|¥|₧|ƒ|ª|º|¿|⌐|¬|½|¼|¡|«|»|░|▒|▓|│|┤|╡|╢|╖|╕|╣|║|╗|╝|╜|╛|┐|└|┴|┬|├|─|┼|╞|╟|╚|╔|╩|╦|╠|═|╬|╧|╨|╤|╥|╙|╘|╒|╓|╫|╪|┘|┌|█|▄|▌|▐|▀|α|ß|Γ|π|Σ|σ|µ|τ|Φ|Ω|δ|∞|φ|ε|∩|≡|±|≥|≤|⌠|⌡|÷|≈|°|\+|∙|·|√|ⁿ|\)|\(|²|■|\~|\.|\?|\*|\!|\✓|\^|？|｡|\。|\|:|;|＂"""



def remove_contractions(text):
    return contractions.fix(text)

def clean_punctuations(text):
    return clean(text, fix_unicode=True, no_punct=True, to_ascii=False)

def simplifiy_chinese(text):
    return chinese_converter.to_simplified(text)

def preprocessing(df, col_name):
    df = \
    df.withColumn(col_name, F.lower(col(col_name)))\
    .withColumn(col_name, F.regexp_replace(col_name, linebreak_pattern, '')) \
    .withColumn(col_name, F.regexp_replace(col_name, r'@[\w]+', 'ALIAS'))\
    .withColumn(col_name, F.regexp_replace(col_name, phone_pattern, 'PHONE'))\
    .withColumn(col_name, F.regexp_replace(col_name, key_pattern, '')) \
    .withColumn(col_name, F.regexp_replace(col_name, "#([a-zA-Z0-9_]{1,50})",''))\
    .withColumn(col_name, F.regexp_replace(col_name, r'\s+', ' ')) \
    .withColumn(col_name, F.regexp_replace(col_name, num, '')) \
    .withColumn(col_name, F.regexp_replace(col_name, "px|",''))\
    .withColumn(col_name, noContr(F.col(col_name))) \
    .filter( (df[col_name] != "") | (df[col_name] != None)) \
    .withColumn(col_name, sentences(F.col(col_name))) \
    .select(flatten(F.col(col_name)).alias('text')) \
    .rdd.map(lambda x : (" ".join(x.text), 1)).toDF(["text","id"]).drop("id") \
    .withColumn(col_name, F.regexp_replace(col_name, symbols, '')) \
    .withColumn(col_name,cleanPunctUDF(F.col(col_name))) \
    

    return df.filter(F.col(col_name) != "")


cleanPunctUDF = udf(lambda x : clean_punctuations(x), StringType())

#### 3. PREPROCESSING I - CHINESE

In [42]:
c.show(100, False)


# 🆂🅰🅼🅿🅻🅴
## CLICK TO SHOW BEFORE PREPROCESSING I

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                      |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|px|缩略图|南蓮園池 南蓮園池位於香港九龍黃大仙區鑽石山鳳德道號，康樂及文化事務署轄下公園，以每年元的象徵式的費用委託志蓮淨苑管理、營運和保育公園。                                                                                         |
|此外，志蓮淨苑亦負責公園的設計、監督建造及日常維修工作。                                                                                             

In [95]:
import chinese_converter
from pyspark.sql.types import StringType, ArrayType
from pyspark.sql.functions import col, udf, size, split

def simplifiy_chinese(text):
    return chinese_converter.to_simplified(text)

udfSimpChinese = udf(lambda x : simplifiy_chinese(x), StringType())

In [96]:
pChi = preprocessing(c, "text")
pChi = pChi.withColumn("text", udfSimpChinese(F.col("text")))
pChi.show(500, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK TO SHOW AFTER PREPROCESSING I

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                           

In [98]:
write_avro(pChi, "output/3_PP_CHI")

#### 3. PREPROCESSING I - ENGLISH

In [6]:
i.show(100, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE THE UNPROCESSED ENGLISH SENTENCES

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [7]:
i = i.withColumnRenamed("col", "text")

In [8]:
from pyspark.sql.functions import *
def remove_contractions(text):
    return contractions.fix(text)

noContr = udf(lambda x : remove_contractions(x), StringType())

def remove_contr(df, colname):
    return df.withColumn(colname, col(colname)).withColumn(colname, noContr(col(colname)))

In [9]:
## remove contrations before clear punctuations

pEng = preprocessing(remove_contr(i, "text"), "text")
pEng.show(500, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE THE PROCESSED ENGLISH SENTENCES

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
pEng.write.mode("overwrite").format("avro").save("output/3_PP_ENG")

#### 3. PREPROCESSING I - MALAY

In [103]:
m = m.withColumnRenamed("col", "text")

In [104]:
m.show(100, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE THE UNPROCESSED ENGLISH SENTENCES

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                                       

In [105]:
pBM = preprocessing(m, "text")
pBM.show(500, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE THE PROCESSED ENGLISH SENTENCES

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [106]:
write_avro(pBM, "output/3_PP_BM")

#### 4. PREPROCESSING II - ENGLISH

In [16]:
bc = spark.read.format("avro").load("output/3_PP_CHI")
bi = spark.read.format("avro").load("output/3_PP_ENG")
bm = spark.read.format("avro").load("output/3_PP_BM")

In [21]:
from pyspark.sql.functions import sentences, lit, flatten

def get_sentence(df, colname):
    return df.select(sentences(df[colname]).alias("text"))

sen = get_sentence(bi, "text")
flat = sen.select(flatten(sen.text).alias('text'))
a = flat.rdd.map(lambda x : (' '.join(x[0]),1)).toDF(["text", "id"]).drop("id")
a.show(100, False)


# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE THE FINAL RESULT OF ENGLISH

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                       

In [22]:
write_avro(a, "output/4_ENG_FINAL")

In [19]:
sen = get_sentence(bc, "text")
flat = sen.select(flatten(sen.text).alias('text'))
a = flat.rdd.map(lambda x : (' '.join(x[0]),1)).toDF(["text", "id"]).drop("id")
a.show(100, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE THE FINAL RESULT OF CHINESE

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [20]:
write_avro(a, "output/4_CHI_FINAL")

In [23]:
sen = get_sentence(bm, "text")
flat = sen.select(flatten(sen.text).alias('text'))
a = flat.rdd.map(lambda x : (' '.join(x[0]),1)).toDF(["text", "id"]).drop("id")
a.show(100, False)

# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE THE FINAL RESULT OF MALAY

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                                                       

In [25]:
write_avro(a, "output/4_MALAY_FINAL")